In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

num_classes = 10
batch_size = 100
num_epochs = 10

class MNIST_Data:
	def __init__(self):
		(x_train, y_train), (x_test, y_test) = mnist.load_data()
		self.dim_img_x = 28
		self.dim_img_y = 28
		self.x_train = x_train.reshape(x_train.shape[0], self.dim_img_x, self.dim_img_y, 1)/255.0
		self.y_train = keras.utils.to_categorical(y_train, num_classes)
		self.x_test = x_test.reshape(x_test.shape[0], self.dim_img_x, self.dim_img_y, 1)/255.0
		self.y_test = keras.utils.to_categorical(y_test, num_classes)
	def train(self):
		return self.x_train, self.y_train
	def test(self):
		return self.x_test, self.y_test
	def input_shape(self):
		return (self.dim_img_x, self.dim_img_y, 1)
    
class generalConvModel:
	def __init__(self):
		self.model = Sequential()
	
	def add_conv_layer(self, num_filters, kernel_size):
		self.model.add(Conv2D(filters = num_filters, kernel_size = kernel_size,
				activation = 'relu', input_shape = input_shape))
	def add_max_pool_layer(self, pool_size):
		self.model.add(MaxPooling2D(pool_size = pool_size))
	def dropout(self, dropout_rate):
		self.model.add(Dropout(dropout_rate))
	def add_dense_layer(self, output_dim, l2_coeff, activation):
		self.model.add(Dense(output_dim, kernel_regularizer = l2(l2_coeff),
				activation = activation))
	def flatten(self):
		self.model.add(Flatten())
	
	def compile_model(self):
		self.model.compile(loss=keras.losses.categorical_crossentropy,
        		optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
	def fit_model_w_val_set(self, x_train, y_train, x_val, y_val):
		self.model.fit(x_train, y_train,
				batch_size=batch_size, epochs=num_epochs,
				verbose=True, validation_data=(x_val, y_val))
	def train_model(self, x_train, y_train):
		self.model.fit(x_train, y_train,
				batch_size = batch_size, epochs = num_epochs,
				verbose = True)
	def evaluate_model(self, x_test, y_test):
		self.score = self.model.evaluate(x_test, y_test, verbose=False)
		print('Test loss:', self.score[0])
		print('Test accuracy:', self.score[1])
    
data = MNIST_Data()
x_train_val, y_train_val = data.train()
x_test, y_test = data.test()
input_shape = data.input_shape()

Using TensorFlow backend.


In [2]:
#Low parameter model
def build_low_param_graph(model):
	model.add_conv_layer(1, (2,2))
	model.add_max_pool_layer((2,2))
	model.dropout(0.1)
	model.flatten()
	model.add_dense_layer(num_classes, 0.01, 'softmax')
	return model
#Tuning Hyperparameters
'''
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size = 0.25)
model = generalConvModel()
model = build_low_param_graph(model)
model.compile_model()
model.fit_model_w_val_set(x_train, y_train, x_val, y_val)
'''
low_param_model = generalConvModel()
low_param_model = build_low_param_graph(low_param_model)
low_param_model.compile_model()
low_param_model.train_model(x_train_val, y_train_val)
low_param_model.evaluate_model(x_test, y_test)

Epoch 1/10
60000/60000 [==============================] - 9s 147us/step - loss: 1.2428 - acc: 0.7184
Epoch 2/10
60000/60000 [==============================] - 7s 117us/step - loss: 0.6295 - acc: 0.8656
Epoch 3/10
60000/60000 [==============================] - 7s 113us/step - loss: 0.5483 - acc: 0.8733
Epoch 4/10
60000/60000 [==============================] - 7s 109us/step - loss: 0.5088 - acc: 0.8773
Epoch 5/10
60000/60000 [==============================] - 6s 108us/step - loss: 0.4820 - acc: 0.8802
Epoch 6/10
60000/60000 [==============================] - 7s 109us/step - loss: 0.4662 - acc: 0.8819
Epoch 7/10
60000/60000 [==============================] - 6s 108us/step - loss: 0.4546 - acc: 0.8828
Epoch 8/10
60000/60000 [==============================] - 7s 114us/step - loss: 0.4434 - acc: 0.8828
Epoch 9/10
60000/60000 [==============================] - 8s 131us/step - loss: 0.4374 - acc: 0.8829
Epoch 10/10
60000/60000 [==============================] - 8s 141us/step - loss: 0.4299 - a

In [3]:
#Deeper Model
def build_graph(model):
	model.add_conv_layer(10, (5,5))
	model.add_max_pool_layer((2,2))
	model.dropout(0.1)
	model.add_conv_layer(10, (3,3))
	model.add_max_pool_layer((2,2))
	model.dropout(0.1)
	model.flatten()
	model.add_dense_layer(20, 0.01, 'relu')
	model.add_dense_layer(num_classes, 0, 'softmax')
	return model

#Tuning Hyperparameters
'''
model = generalConvModel()
model = build_graph(model)
model.compile_model()
model.fit_model_w_val_set(x_train, y_train, x_val, y_val)
'''
deeper_model = generalConvModel()
deeper_model = build_graph(deeper_model)
deeper_model.compile_model()
deeper_model.train_model(x_train_val, y_train_val)
deeper_model.evaluate_model(x_test, y_test)

Epoch 1/10
60000/60000 [==============================] - 28s 468us/step - loss: 0.7175 - acc: 0.8287
Epoch 2/10
60000/60000 [==============================] - 26s 435us/step - loss: 0.2524 - acc: 0.9512
Epoch 3/10
60000/60000 [==============================] - 26s 433us/step - loss: 0.1799 - acc: 0.9632
Epoch 4/10
60000/60000 [==============================] - 29s 483us/step - loss: 0.1505 - acc: 0.9669
Epoch 5/10
60000/60000 [==============================] - 28s 460us/step - loss: 0.1332 - acc: 0.9713
Epoch 6/10
60000/60000 [==============================] - 28s 464us/step - loss: 0.1244 - acc: 0.9732
Epoch 7/10
60000/60000 [==============================] - 29s 479us/step - loss: 0.1156 - acc: 0.9746
Epoch 8/10
60000/60000 [==============================] - 36s 607us/step - loss: 0.1089 - acc: 0.9766
Epoch 9/10
60000/60000 [==============================] - 29s 477us/step - loss: 0.1062 - acc: 0.9768
Epoch 10/10
60000/60000 [==============================] - 29s 479us/step - loss: 